In [187]:
import torch
import torch.nn as nn
import pandas as pd

# Abstraction Level 0

In [168]:
component = nn.LSTM(input_size=4,hidden_size=4,num_layers=4,batch_first=True)
loss_function = torch.nn.MSELoss()
optimizer = torch.optim.AdamW(component.parameters(), lr=0.02)
component.train()

LSTM(4, 4, num_layers=4, batch_first=True)

In [169]:
cs1= 0
cs2 = 0.8
cs3 = 0.1
cs4 = 0.1
previous_condition_state = torch.tensor([[cs1,cs2,cs3,cs4]])

In [170]:
predicted_condition_state, (longterm,shorterm) = component(previous_condition_state)

In [171]:
print('predicted_condition_state:\n',predicted_condition_state, '\n\nShorterm:\n',shorterm,'\n\nLongterm:\n',longterm)

predicted_condition_state:
 tensor([[ 0.0591, -0.1569,  0.0081,  0.1124]], grad_fn=<SqueezeBackward1>) 

Shorterm:
 tensor([[-0.4282, -0.1888,  0.2147,  0.3445],
        [-0.0771, -0.1660, -0.4031,  0.0725],
        [-0.0911,  0.1581, -0.0744,  0.1901],
        [ 0.1808, -0.2432,  0.0146,  0.2261]], grad_fn=<SqueezeBackward1>) 

Longterm:
 tensor([[-0.2487, -0.1121,  0.0847,  0.1362],
        [-0.0310, -0.0793, -0.1935,  0.0317],
        [-0.0517,  0.1025, -0.0402,  0.0994],
        [ 0.0591, -0.1569,  0.0081,  0.1124]], grad_fn=<SqueezeBackward1>)


In [172]:
# Next stage:
cs1= 0
cs2 = 0.7
cs3 = 0.2
cs4 = 0.1
actual_condition_state = torch.tensor([[cs1,cs2,cs3,cs4]])
print('previous condition state:',previous_condition_state)
print('predicted condition state:',predicted_condition_state)
print('actual condtion state:',actual_condition_state)

previous condition state: tensor([[0.0000, 0.8000, 0.1000, 0.1000]])
predicted condition state: tensor([[ 0.0591, -0.1569,  0.0081,  0.1124]], grad_fn=<SqueezeBackward1>)
actual condtion state: tensor([[0.0000, 0.7000, 0.2000, 0.1000]])


In [173]:
loss = loss_function(predicted_condition_state,actual_condition_state)

In [174]:
print(loss.item())

0.19367893040180206


In [175]:
optimizer.zero_grad()
loss.backward()
optimizer.step()

In [176]:
# model_predict_next_state
predicted_condition_state,(longterm,shorterm) = component(actual_condition_state,
                                                          (longterm.detach(),shorterm.detach()))

In [177]:
# Next stage:
cs1= 0
cs2 = 0.6
cs3 = 0.2
cs4 = 0.2
previous_condition_state = actual_condition_state
actual_condition_state = torch.tensor([[cs1,cs2,cs3,cs4]])
print('previous condition state:',previous_condition_state)
print('predicted condition state:',predicted_condition_state)
print('actual condtion state:',actual_condition_state)

previous condition state: tensor([[0.0000, 0.7000, 0.2000, 0.1000]])
predicted condition state: tensor([[ 0.0885, -0.1925,  0.0589,  0.1280]], grad_fn=<SqueezeBackward1>)
actual condtion state: tensor([[0.0000, 0.6000, 0.2000, 0.2000]])


In [178]:
loss = loss_function(predicted_condition_state,actual_condition_state)

In [179]:
print(loss.item())

0.16524168848991394


In [166]:
optimizer.zero_grad()
loss.backward(retain_graph=True)
optimizer.step()

In [180]:
# model_predict_next_state
predicted_condition_state,(longterm,shorterm) = component(actual_condition_state,
                                                          (longterm.detach(),shorterm.detach()))

In [181]:
# Next stage:
cs1= 0
cs2 = 0.5
cs3 = 0.2
cs4 = 0.3
previous_condition_state = actual_condition_state
actual_condition_state = torch.tensor([[cs1,cs2,cs3,cs4]])
print('previous condition state:',previous_condition_state)
print('predicted condition state:',predicted_condition_state)
print('actual condtion state:',actual_condition_state)

previous condition state: tensor([[0.0000, 0.6000, 0.2000, 0.2000]])
predicted condition state: tensor([[ 0.1094, -0.2013,  0.0812,  0.1329]], grad_fn=<SqueezeBackward1>)
actual condtion state: tensor([[0.0000, 0.5000, 0.2000, 0.3000]])


In [182]:
loss = loss_function(predicted_condition_state,actual_condition_state)

In [183]:
print(loss.item())

0.13644912838935852


In [184]:
optimizer.zero_grad()
loss.backward(retain_graph=True)
optimizer.step()

# Generalisation at Component Level

In [204]:
row_1 = {'current_state': [0,0.8,0.1,0.1], 'next_state': [0,0.7,0.2,0.1]}
row_2 = {'current_state': [0,0.7,0.2,0.1], 'next_state': [0,0.6,0.2,0.2]}
row_3 = {'current_state': [0,0.6,0.2,0.2], 'next_state': [0,0.5,0.2,0.3]}
row_4 = {'current_state': [0,0.5,0.2,0.3], 'next_state': [0,0.4,0.3,0.3]}
row_5 = {'current_state': [0,0.4,0.3,0.3], 'next_state': [0,0.2,0.3,0.5]}
data = [row_1,row_2,row_3,row_4,row_5]

In [205]:
df = pd.DataFrame(data)

In [206]:
df

,current_state,next_state
0,"[0, 0.8, 0.1, 0.1]","[0, 0.7, 0.2, 0.1]"
1,"[0, 0.7, 0.2, 0.1]","[0, 0.6, 0.2, 0.2]"
2,"[0, 0.6, 0.2, 0.2]","[0, 0.5, 0.2, 0.3]"
3,"[0, 0.5, 0.2, 0.3]","[0, 0.4, 0.3, 0.3]"
4,"[0, 0.4, 0.3, 0.3]","[0, 0.2, 0.3, 0.5]"


In [207]:
component = nn.LSTM(input_size=4,hidden_size=4,num_layers=4,batch_first=True)
loss_function = torch.nn.MSELoss()
optimizer = torch.optim.AdamW(component.parameters(), lr=0.02)
component.train()

LSTM(4, 4, num_layers=4, batch_first=True)

In [208]:
step = 0
for epoch in range(3):
    for i,r in df.iterrows():
        inputs = torch.tensor([r['current_state']])
        target = torch.tensor([r['next_state']])
        if step == 0:
            prediction, (long_term,short_term) = component(inputs)
        else:
            prediction, (long_term,short_term) = component(inputs,(long_term.detach(),short_term.detach()))
        loss = loss_function(prediction,target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_value = round(loss.item(),3)
        step += 1
        print('training_step',step,';loss:',loss_value,'\n')

training_step 1 ;loss: 0.13 

training_step 2 ;loss: 0.106 

training_step 3 ;loss: 0.088 

training_step 4 ;loss: 0.061 

training_step 5 ;loss: 0.061 

training_step 6 ;loss: 0.107 

training_step 7 ;loss: 0.072 

training_step 8 ;loss: 0.046 

training_step 9 ;loss: 0.018 

training_step 10 ;loss: 0.021 

training_step 11 ;loss: 0.058 

training_step 12 ;loss: 0.025 

training_step 13 ;loss: 0.006 

training_step 14 ;loss: 0.002 

training_step 15 ;loss: 0.024 



In [212]:
# Save model
torch.save(component.state_dict(),'component_type_A.pth')

In [226]:
# Save the long_term and short_term separately:
row1 = {'City': '13E',
        'S': 123, 
        'G':'A1', 
        'Component':'A50',
        'latest_condition':inputs.detach().numpy(),
        'saved_model_path': 'component_type_A.pth',
        'long_term_memory':long_term.detach().numpy(),
        'short_term_memory':short_term.detach().numpy()}
simulation_data = pd.DataFrame([row1])

In [227]:
simulation_data

,City,S,G,Component,latest_condition,saved_model_path,long_term_memory,short_term_memory
0,13E,123,A1,A50,"[[0.0, 0.4, 0.3, 0.3]]",component_type_A.pth,"[[0.3730696, -0.076551, 0.52359045, -0.0314048...","[[0.88893974, -0.26120937, 1.2495234, -0.09117..."


# Generalisation at Group Level

In [232]:
group = nn.LSTM(input_size=4,hidden_size=4,num_layers=4,batch_first=True)
loss_function = torch.nn.MSELoss()
optimizer = torch.optim.AdamW(group.parameters(), lr=0.02)
group.train()

inputs_group_A = torch.tensor(df['current_state'].values.tolist())
targets_group_A = torch.tensor(df['next_state'].values.tolist())

In [233]:
inputs_group_A

tensor([[0.0000, 0.8000, 0.1000, 0.1000],
        [0.0000, 0.7000, 0.2000, 0.1000],
        [0.0000, 0.6000, 0.2000, 0.2000],
        [0.0000, 0.5000, 0.2000, 0.3000],
        [0.0000, 0.4000, 0.3000, 0.3000]])

In [234]:
targets_group_A

tensor([[0.0000, 0.7000, 0.2000, 0.1000],
        [0.0000, 0.6000, 0.2000, 0.2000],
        [0.0000, 0.5000, 0.2000, 0.3000],
        [0.0000, 0.4000, 0.3000, 0.3000],
        [0.0000, 0.2000, 0.3000, 0.5000]])

In [258]:
step = 0
for epoch in range(10):
    output, (h,c) = group(inputs_group_A)
    loss = loss_function(output,targets_group_A)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(loss.item())

# Save model
# Torch.save(group.state_dict(),'group_type_A.pth')
# Repeat the same saving process

0.1479625403881073
0.12653174996376038
0.10725084692239761
0.09027619659900665
0.07606229186058044
0.0647188350558281
0.05624906346201897
0.05033007264137268
0.04627736285328865
0.04340410977602005


# Generalisation At Struct Level

In [272]:
struct = nn.LSTM(input_size=4,hidden_size=4,num_layers=4,batch_first=True)
loss_function = torch.nn.MSELoss()
optimizer = torch.optim.AdamW(struct.parameters(), lr=0.02)
struct.train()

LSTM(4, 4, num_layers=4, batch_first=True)

In [273]:
input_structure_A = [inputs_group_A,inputs_group_A,inputs_group_A,inputs_group_A]
target_structure_A = [targets_group_A,targets_group_A,targets_group_A,targets_group_A]
input_structure_A = torch.stack(input_structure_A)
target_structure_A = torch.stack(target_structure_A)

In [274]:
input_structure_A

tensor([[[0.0000, 0.8000, 0.1000, 0.1000],
         [0.0000, 0.7000, 0.2000, 0.1000],
         [0.0000, 0.6000, 0.2000, 0.2000],
         [0.0000, 0.5000, 0.2000, 0.3000],
         [0.0000, 0.4000, 0.3000, 0.3000]],

        [[0.0000, 0.8000, 0.1000, 0.1000],
         [0.0000, 0.7000, 0.2000, 0.1000],
         [0.0000, 0.6000, 0.2000, 0.2000],
         [0.0000, 0.5000, 0.2000, 0.3000],
         [0.0000, 0.4000, 0.3000, 0.3000]],

        [[0.0000, 0.8000, 0.1000, 0.1000],
         [0.0000, 0.7000, 0.2000, 0.1000],
         [0.0000, 0.6000, 0.2000, 0.2000],
         [0.0000, 0.5000, 0.2000, 0.3000],
         [0.0000, 0.4000, 0.3000, 0.3000]],

        [[0.0000, 0.8000, 0.1000, 0.1000],
         [0.0000, 0.7000, 0.2000, 0.1000],
         [0.0000, 0.6000, 0.2000, 0.2000],
         [0.0000, 0.5000, 0.2000, 0.3000],
         [0.0000, 0.4000, 0.3000, 0.3000]]])

In [276]:
step = 0
for epoch in range(10):
    output, (h,c) = struct(input_structure_A)
    loss = loss_function(output,target_structure_A)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(loss.item())

0.09143637865781784
0.07891888171434402
0.06808613240718842
0.058878619223833084
0.051082588732242584
0.044479988515377045
0.038846008479595184
0.034010179340839386
0.02987174689769745
0.026354685425758362


In [271]:
# Save model for each structure
# Torch.save(struct.state_dict(),'struct_123123.pth')
# save long term and short term

# Generalisation At City, State, Country Level

In [2]:
# just dont change the structure datatable
# city_input = [input_structure_A, input_structure_B, input_structure_C]
# city_target = [target_structure_A, target_structure_B, target_structure_C]
# Rinse and repeat
# There, my vision!